# Comparación y corrección de texto

* *60:00 min* | Última modificación: Agosto 11, 2021 | YouTube

In [3]:
!pip3 install --quiet fuzzywuzzy python-Levenshtein

In [12]:
import pandas as pd
from fuzzywuzzy import fuzz, process

pd.set_option("display.notebook_repr_html", False)

## Comparación de cadenas de texto

In [13]:
#
# Comparación de strings
#
display(
    fuzz.WRatio('hola mundo', 'hola mundo'),
    fuzz.WRatio('hola mundo', 'Hola mundo'),
    fuzz.WRatio('hola mundo', 'Hola Mundo'),
    fuzz.WRatio('hola mundo', 'hola mundo!')
)

100

100

100

100

In [14]:
#
# Comparación de strings
#
display(
    fuzz.WRatio('hola mundo!', 'hola mundo'),
    fuzz.WRatio('hola mundo', 'Hola mundo!'),
    fuzz.WRatio('hola mundo', 'Hola mundoo'),
    fuzz.WRatio('hola mundo', 'hola mundooo'),
    fuzz.WRatio('hola mundo', 'hola mundoooo'),
    fuzz.WRatio('hola mundo', 'hola'),
    fuzz.WRatio('hola', 'hola mundo')
)

100

100

95

91

87

90

90

In [15]:
#
# Comparación con arrays
#
base_string = "hola mundo"
array = pd.Series(
    [
        "Hola mundo",
        "mundo",
        "Hola",
        "Hola mundo cruel!",
    ]
)


process.extract(
    base_string,
    array,
)

[('Hola mundo', 100, 0),
 ('mundo', 90, 1),
 ('Hola', 90, 2),
 ('Hola mundo cruel!', 90, 3)]

## Corrección y unificación básica cuando se conoce la clave

In [16]:
%%writefile /tmp/data.csv
ciudad
Medellin
Bogota
bogota
bogote
Mdellin
medellin
Cali
cali
CALI

Overwriting /tmp/data.csv


In [17]:
df = pd.read_csv('/tmp/data.csv')
df

     ciudad
0  Medellin
1    Bogota
2    bogota
3    bogote
4   Mdellin
5  medellin
6      Cali
7      cali
8      CALI

In [18]:
#
# Nombres validos
#
valid_names = ["Medellin", "Bogota", "Cali"]
df["ciudad_"] = df.ciudad.copy()

#
# Valor mínimo de similitud para hacer el cambio
#
min_threshold = 80

#
# Estructuta básica
#
for valid_name in valid_names:

    potential_matches = process.extract(
        valid_name,
        df.ciudad,
        limit=df.shape[0],
    )

    for potential_match in potential_matches:

        if potential_match[1] >= min_threshold:

            df.loc[df.ciudad == potential_match[0], "ciudad_"] = valid_name

df

     ciudad   ciudad_
0  Medellin  Medellin
1    Bogota    Bogota
2    bogota    Bogota
3    bogote    Bogota
4   Mdellin  Medellin
5  medellin  Medellin
6      Cali      Cali
7      cali      Cali
8      CALI      Cali